In [1]:
import pandas as pd
import numpy as np
import math
from graphviz import Digraph

In [2]:
def calculate_entropy(data, target_col):
    """
    Calculate entropy of the target column
    """
    values = data[target_col].value_counts(normalize=True)
    entropy = -sum(values * np.log2(values))
    return round(entropy, 3)

In [3]:
def calculate_information_gain(data, feature_col, target_col):
    """
    Calculate information gain of a feature
    """
    total_entropy = calculate_entropy(data, target_col)
    values = data[feature_col].unique()
    weighted_entropy = 0

    for value in values:
        subset = data[data[feature_col] == value]
        weight = len(subset) / len(data)
        subset_entropy = calculate_entropy(subset, target_col)
        weighted_entropy += weight * subset_entropy

    info_gain = total_entropy - weighted_entropy
    return round(info_gain, 3)

In [4]:
def build_decision_tree(data, target_col, features, depth=0):
    """
    Recursively build a decision tree using ID3 algorithm
    """
    # Check if the dataset is homogeneous
    if len(data[target_col].unique()) == 1:
        return data[target_col].iloc[0]

    # If no features left, return the most common class
    if not features:
        return data[target_col].mode()[0]

    # Calculate information gain for each feature
    gains = {feature: calculate_information_gain(data, feature, target_col) for feature in features}
    best_feature = max(gains, key=gains.get)  # Choose the feature with the highest information gain

    # Build the decision tree
    tree = {best_feature: {}}

    for value in data[best_feature].unique():
        subset = data[data[best_feature] == value]
        if subset.empty:
            tree[best_feature][value] = data[target_col].mode()[0]
        else:
            subtree = build_decision_tree(
                subset,
                target_col,
                [feat for feat in features if feat != best_feature],
                depth + 1
            )
            tree[best_feature][value] = subtree

    return tree

In [5]:
def print_rules(tree, path=""):
    """
    Convert a decision tree into rules
    """
    if not isinstance(tree, dict):  # Leaf node
        print(f"{path} => {tree}")
        return

    for node, branches in tree.items():
        for value, subtree in branches.items():
            new_path = f"{path} {node} = {value}" if path else f"{node} = {value}"
            print_rules(subtree, new_path)

In [6]:
def visualize_tree(tree, graph=None, parent=None, edge_label=""):
    """
    Visualize a decision tree using Graphviz
    """
    if graph is None:
        graph = Digraph(format='png')
        graph.attr('node', shape='ellipse')

    if isinstance(tree, dict):  # Decision node
        for node, branches in tree.items():
            graph.node(node, label=node)  # Add decision node
            if parent:
                graph.edge(parent, node, label=edge_label)
            for branch_value, subtree in branches.items():
                visualize_tree(subtree, graph, parent=node, edge_label=str(branch_value))
    else:  # Leaf node
        leaf_label = f"{tree}"  # Use leaf value as label
        leaf_node = f"{parent}_{leaf_label}"  # Unique ID for leaf
        graph.node(leaf_node, label=leaf_label, shape='box')
        graph.edge(parent, leaf_node, label=edge_label)

    return graph

In [7]:
# Load dataset
file_path = input("Enter the path to your CSV file: ")

Enter the path to your CSV file: /content/ID3_Dataseti.csv


In [11]:
target_col = input("Enter the target column name: ")

Enter the target column name: Play


In [12]:
data = pd.read_csv(file_path)

In [13]:
# Define features (exclude target column)
features = [col for col in data.columns if col != target_col]

# Build the decision tree
decision_tree = build_decision_tree(data, target_col, features)

# Visualize the decision tree
dot = visualize_tree(decision_tree)
dot.render("decision_tree1")  # Save the tree to a file
print("Decision tree has been saved as decision_tree1.png")
print("Decision Tree Rules:")
print_rules(decision_tree)

Decision tree has been saved as decision_tree1.png
Decision Tree Rules:
Outlook = rainy Humidity = high => no
Outlook = rainy Humidity = normal => yes
Outlook = overcast => yes
Outlook = sunny Windy = False => yes
Outlook = sunny Windy = True => no


# Full Hali

In [ ]:
import pandas as pd
import numpy as np
import math
from graphviz import Digraph

def calculate_entropy(data, target_col):
    """
    Calculate entropy of the target column
    """
    values = data[target_col].value_counts(normalize=True)
    entropy = -sum(values * np.log2(values))
    return round(entropy, 3)

def calculate_information_gain(data, feature_col, target_col):
    """
    Calculate information gain of a feature
    """
    total_entropy = calculate_entropy(data, target_col)
    values = data[feature_col].unique()
    weighted_entropy = 0

    for value in values:
        subset = data[data[feature_col] == value]
        weight = len(subset) / len(data)
        subset_entropy = calculate_entropy(subset, target_col)
        weighted_entropy += weight * subset_entropy

    info_gain = total_entropy - weighted_entropy
    return round(info_gain, 3)

def build_decision_tree(data, target_col, features, depth=0):
    """
    Recursively build a decision tree using ID3 algorithm
    """
    # Check if the dataset is homogeneous
    if len(data[target_col].unique()) == 1:
        return data[target_col].iloc[0]

    # If no features left, return the most common class
    if not features:
        return data[target_col].mode()[0]

    # Calculate information gain for each feature
    gains = {feature: calculate_information_gain(data, feature, target_col) for feature in features}
    best_feature = max(gains, key=gains.get)  # Choose the feature with the highest information gain

    # Build the decision tree
    tree = {best_feature: {}}

    for value in data[best_feature].unique():
        subset = data[data[best_feature] == value]
        if subset.empty:
            tree[best_feature][value] = data[target_col].mode()[0]
        else:
            subtree = build_decision_tree(
                subset,
                target_col,
                [feat for feat in features if feat != best_feature],
                depth + 1
            )
            tree[best_feature][value] = subtree

    return tree

def visualize_tree(tree, graph=None, parent=None, edge_label=""):
    """
    Visualize a decision tree using Graphviz
    """
    if graph is None:
        graph = Digraph(format='png')
        graph.attr('node', shape='ellipse')

    if isinstance(tree, dict):  # Decision node
        for node, branches in tree.items():
            graph.node(node, label=node)  # Add decision node
            if parent:
                graph.edge(parent, node, label=edge_label)
            for branch_value, subtree in branches.items():
                visualize_tree(subtree, graph, parent=node, edge_label=str(branch_value))
    else:  # Leaf node
        leaf_label = f"{tree}"  # Use leaf value as label
        leaf_node = f"{parent}_{leaf_label}"  # Unique ID for leaf
        graph.node(leaf_node, label=leaf_label, shape='box')
        graph.edge(parent, leaf_node, label=edge_label)

    return graph

def preprocess_data(data):
    """
    Preprocess the dataset:
    1. Drop numeric columns (warn user if any are found).
    2. Check for missing values and handle them.
    3. Verify target column and feature columns.
    """
    # Detect numeric columns
    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    if numeric_columns:
        print(f"Warning: The dataset contains numeric columns: {numeric_columns}")
        print("These columns will be dropped.")
        data = data.drop(columns=numeric_columns)

    # Handle missing values
    if data.isnull().values.any():
        print("Warning: Missing values detected in the dataset.")
        print("Filling missing values with the most frequent value in each column.")
        data = data.fillna(data.mode().iloc[0])

    return data

# Load dataset
file_path = input("Enter the path to your CSV file: ")
target_col = input("Enter the target column name: ")
data = pd.read_csv(file_path)

# Preprocess the dataset
data = preprocess_data(data)

# Check if target column exists
if target_col not in data.columns:
    raise ValueError(f"Target column '{target_col}' not found in the dataset.")

# Define features (exclude target column)
features = [col for col in data.columns if col != target_col]

# Build the decision tree
decision_tree = build_decision_tree(data, target_col, features)

# Visualize the decision tree
dot = visualize_tree(decision_tree)
dot.render("decision_tree1")  # Save the tree to a file
print("Decision tree has been saved as decision_tree1.png")

# Performance Metrics
print("\nPerformance Metrics:")
print(f"Number of features used: {len(features)}")
print(f"Number of classes in target column: {data[target_col].nunique()}")
print(f"Dataset size: {data.shape[0]} rows, {data.shape[1]} columns")